In [23]:
import wntr

wn = wntr.network.WaterNetworkModel('Net1.inp')
print("Loaded successfully")


Loaded successfully


In [24]:
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

print("Baseline simulation completed")

Baseline simulation completed


In [25]:
pressure_baseline = results.node['pressure']


In [26]:
pressure_baseline.head()


name,10,11,12,13,21,22,23,31,32,9,2
0,89.717087,83.890205,82.317276,83.476387,82.767410,83.539085,84.931053,81.500954,77.934105,0.0,36.575996
3600,90.454926,84.729393,83.251549,84.406586,83.667519,84.464272,85.857826,82.406822,78.848099,0.0,37.511158
7200,90.917648,85.255745,84.155159,85.054062,83.953423,84.992241,86.371002,82.307472,78.707024,0.0,38.424873
10800,91.417061,85.823898,84.786301,85.681648,84.562263,85.614861,86.994759,82.919395,79.322929,0.0,39.056461
14400,91.644539,86.082703,85.396606,85.978691,84.503044,85.736961,87.100243,82.392494,78.709198,0.0,39.673317


In [27]:
P0 = pressure_baseline.iloc[0]


In [28]:
print("Min pressure:", P0.min())
print("Max pressure:", P0.max())


Min pressure: 0.0
Max pressure: 89.71709


In [29]:
import pandas as pd

P0.to_csv("baseline_pressure.csv")


In [30]:
wn = wntr.network.WaterNetworkModel('Net1.inp')

In [31]:
leak_node = '11'   # use one you see in the columns


In [32]:
junction = wn.get_node(leak_node)
junction.add_demand(base=0.01, pattern_name='1')


In [33]:
sim_leak = wntr.sim.EpanetSimulator(wn)
results_leak = sim_leak.run_sim()

pressure_leak = results_leak.node['pressure']
P_leak = pressure_leak.iloc[0]

In [34]:
delta_P = P0 - P_leak

In [35]:
delta_P.sort_values(ascending=False).head(10)

name
11    0.753883
10    0.662971
21    0.274864
31    0.231071
32    0.170410
22    0.083122
23    0.070595
13    0.040459
12    0.006744
9     0.000000
Name: 0, dtype: float32

In [36]:
node_list = list(P0.index)
node_list

['10', '11', '12', '13', '21', '22', '23', '31', '32', '9', '2']

In [37]:
X_sample = delta_P.loc[node_list].values
y_sample = node_list.index(leak_node)

In [38]:
baseline_pressure = pressure_baseline


In [39]:
import numpy as np
import wntr

X = []
y = []

# fixed node order (features)
node_list = list(P0.index)

# valid leak locations
wn_tmp = wntr.network.WaterNetworkModel('Net1.inp')
junction_list = wn_tmp.junction_name_list

# leak sizes
leak_magnitudes = np.linspace(0.002, 0.04, 12)

# time steps (from EPANET results)
time_steps = pressure_baseline.index.tolist()

for leak_node in junction_list:
    for leak_size in leak_magnitudes:
        
        # reload clean network
        wn = wntr.network.WaterNetworkModel('Net1.inp')
        wn.get_node(leak_node).add_demand(leak_size, '1')
        
        # simulate
        sim = wntr.sim.EpanetSimulator(wn)
        results = sim.run_sim()
        
        pressure_leak = results.node['pressure']
        
        for t in time_steps:
            delta_P = np.abs(baseline_pressure.loc[t] - pressure_leak.loc[t])
            
            X.append(delta_P.loc[node_list].values)
            y.append(junction_list.index(leak_node))

X = np.array(X)
y = np.array(y)

print("Final dataset shape:", X.shape)
print("Labels shape:", y.shape)

Final dataset shape: (2700, 11)
Labels shape: (2700,)


In [40]:
np.save("X_leak_large.npy", X)
np.save("y_leak_large.npy", y)

In [42]:
import numpy as np

i = np.random.randint(len(X))
top_idx = np.argsort(np.abs(X[i]))[-5:]

print("Top affected nodes:", [node_list[j] for j in top_idx])
print("ΔP values:", X[i][top_idx])

Top affected nodes: ['13', '10', '11', '12', '2']
ΔP values: [1.953 1.963 1.963 1.991 1.993]


In [43]:
import wntr
import networkx as nx

wn = wntr.network.WaterNetworkModel('Net1.inp')
G = wn.get_graph()

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 11
Number of edges: 13


C:\Users\vedik\AppData\Local\Temp\ipykernel_27760\3023024140.py:5: DeprecationWarning: wntr.network.WaterNetworkModel.get_graph is deprecated, use wntr.network.WaterNetworkModel.to_graph instead
  G = wn.get_graph()


In [44]:
node_list = list(G.nodes())
node_to_idx = {node: i for i, node in enumerate(node_list)}

print(node_list)

['10', '11', '12', '13', '21', '22', '23', '31', '32', '9', '2']


In [45]:
edge_index = []

for u, v in G.edges():
    edge_index.append([node_to_idx[u], node_to_idx[v]])
    edge_index.append([node_to_idx[v], node_to_idx[u]])  # undirected graph

edge_index = np.array(edge_index).T

print("edge_index shape:", edge_index.shape)

edge_index shape: (2, 26)


In [46]:
print("First 5 edges:")
for i in range(5):
    src = node_list[edge_index[0, i]]
    dst = node_list[edge_index[1, i]]
    print(src, "->", dst)

First 5 edges:
10 -> 11
11 -> 10
11 -> 12
12 -> 11
11 -> 21


In [47]:
import numpy as np

def make_node_label(leak_idx, num_nodes):
    y_node = np.zeros(num_nodes, dtype=int)
    y_node[leak_idx] = 1
    return y_node

In [48]:
graph_samples = []

num_nodes = X.shape[1]

for i in range(len(X)):
    x_node = X[i].reshape(num_nodes, 1)   # (num_nodes, 1 feature)
    y_node = make_node_label(y[i], num_nodes)
    
    graph_samples.append({
        "x": x_node,
        "edge_index": edge_index,
        "y": y_node
    })

print("Number of graph samples:", len(graph_samples))

Number of graph samples: 2700


In [49]:
sample = graph_samples[0]
print("x shape:", sample["x"].shape)
print("edge_index shape:", sample["edge_index"].shape)
print("y sum (should be 1):", sample["y"].sum())

x shape: (11, 1)
edge_index shape: (2, 26)
y sum (should be 1): 1


In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(X_train.shape, X_test.shape)

(2160, 11) (540, 11)


In [51]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    max_iter=500,
    random_state=42
)

mlp.fit(X_train, y_train)

,hidden_layer_sizes,"(64, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,500
,shuffle,True
,random_state,42


In [52]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = mlp.predict(X_test)

print("Baseline MLP accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Baseline MLP accuracy: 0.85
              precision    recall  f1-score   support

           0       0.83      0.72      0.77        60
           1       0.72      0.80      0.76        60
           2       0.71      0.65      0.68        60
           3       1.00      0.92      0.96        60
           4       0.93      0.90      0.92        60
           5       0.73      0.85      0.78        60
           6       0.85      0.93      0.89        60
           7       0.93      0.93      0.93        60
           8       1.00      0.95      0.97        60

    accuracy                           0.85       540
   macro avg       0.85      0.85      0.85       540
weighted avg       0.85      0.85      0.85       540



In [53]:
import numpy as np

np.save("X_leak_large.npy", X)
np.save("y_leak_large.npy", y)
np.save("edge_index.npy", edge_index)

In [54]:
import numpy as np
print(np.load("X_leak_large.npy").shape)
print(np.load("y_leak_large.npy").shape)
print(np.load("edge_index.npy").shape)

(2700, 11)
(2700,)
(2, 26)
